In [1]:
import sys; sys.path.append("../")
import pandas as pd

from lib.explore import show_unique
from lib.uid import gen_uid
from clean.new_orleans_harbor_pd_cprr import clean as clean_cprr
from clean.new_orleans_harbor_pd_pprr import clean_personnel
from match.new_orleans_harbor_pd import match_uid_with_cprr

In [2]:
cprr = clean_cprr()
cprr

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../clean/new_orleans_harbor_pd_cprr.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "badge_no"] = df.badge_no.str.strip(
../lib/clean.py:134: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.strip().str.lower()\
../lib/clean.py:142: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.strip().str.lower()\


,badge_no,sex,department_desc,rank_desc,rule_code,paragraph_code,recommended_action,action,incident_type,tracking_number,...,occur_month,occur_day,receive_year,receive_month,receive_day,investigation_complete_year,investigation_complete_month,investigation_complete_day,agency,data_production_year
0,HP-56,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2015-3,...,7,5,2015,7,9,2015,8,4,New Orleans Harbor PD,2020
1,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2015-2,...,3,22,2015,4,1,2015,6,29,New Orleans Harbor PD,2020
2,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2016-3,...,6,25,2016,6,27,2016,7,29,New Orleans Harbor PD,2020
3,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2016-5,...,11,20,2016,12,21,2017,1,11,New Orleans Harbor PD,2020
4,HP-33,male,bicycle division,corporal,1.3,2,none,,use of force,2017-7,...,7,15,2017,7,15,2017,8,29,New Orleans Harbor PD,2020
5,HP-56,male,,police officer 2-a,2,3,counselling memorrandum,counselling memmorandum,discourtesy,2017-1,...,12,24,2017,1,10,2017,2,20,New Orleans Harbor PD,2020


In [3]:
pprr = clean_personnel()
pprr

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1842: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
../lib/clean.py:156: FutureWarnin

,first_name,last_name,rank_desc,hourly_salary,middle_initial,term_year,term_month,term_day,hire_year,hire_month,hire_day,pay_effective_year,pay_effective_month,pay_effective_day,uid,agency,data_production_year
0,casey,adams,police sergeant--port,17.00,m,2010,2,28,1980,1,31,1997,4,14,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
1,casey,adams,police sergeant--port,17.68,m,2010,2,28,1980,1,31,2003,12,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
2,casey,adams,police sergeant--port,18.39,m,2010,2,28,1980,1,31,2004,8,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
3,casey,adams,police sergeant--port,19.13,m,2010,2,28,1980,1,31,2005,8,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
4,casey,adams,police sergeant-a,19.13,m,2010,2,28,1980,1,31,2005,8,29,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,brianne,waters,police officer 3-a,21.60,m,2020,5,11,2001,3,5,2015,10,1,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
911,brianne,waters,police sergeant-a,24.34,m,2020,5,11,2001,3,5,2018,5,14,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
912,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
913,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020


In [4]:
matcher = match_uid_with_cprr(cprr, pprr)
matcher.get_sample_pairs()

first_name  \
score_range pair_idx sim_score row_key                                       
1.00-0.95   0        1.0000    c550f567113a27df7d07ea462fb1a5a5      bryan   
                               e6278fa24d4f3a03c7b1fc41f07dd6de      bryan   
            1        1.0000    e0afe1e6553097708bcd87c772cc4141     howard   
                               2e448f6aba55cd31091991480f1af231     howard   
0.95-0.90   0        0.9375    9d9e4d8548e120eee8f80b9a459e7b5a      lamus   
                               cac9967e392f4edf4da41c4e478f96b5      lamus   

                                                                last_name  
score_range pair_idx sim_score row_key                                     
1.00-0.95   0        1.0000    c550f567113a27df7d07ea462fb1a5a5     arena  
                               e6278fa24d4f3a03c7b1fc41f07dd6de     arena  
            1        1.0000    e0afe1e6553097708bcd87c772cc4141  dennison  
                               2e448f6aba55cd31091991480f1af231  dennison  
0.95-0.90   0        0.9375    9d9e4d8548e120eee8f80b9a459e7b5a     lundy  
                               cac9967e392f4edf4da41c4e478f96b5  lundy jr

In [5]:
cprr.mid.unique()

array(['9d9e4d8548e120eee8f80b9a459e7b5a',
       'e0afe1e6553097708bcd87c772cc4141',
       'c550f567113a27df7d07ea462fb1a5a5'], dtype=object)